# 17. Scipy

Scipy implements fundamental algorithms used for scientific computing in Python, including optimization, integration, interpolation, eigenvalue problems, algebraic equations, differential equations, statistics and many other classes of problems.

We will cover a small subset of its functionalities. Refer to https://scipy.org for  the full documentation.

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
import scipy

## 17.1 Integration
scipy offers 2 family of functions to compute definite integrals of functions of a single variable $\int_a^b f(x)\, dx$ depending on whether the integrand is a full python function (*i.e.* we can compute $f(x)$ for any $x$) or we know it only at a given fixed points (*i.e.* we have arrays `X` and `Y = f[X]`).

The general method for the former case is `quad` and that of the later is `trapezoid`.

### `scipy.integrate.quad`

* Basic calling scheme: `res, err = scipy.integrate.quad(f,a,b)`. This will return an approximation of the integral and an *estimate* of the accuracy of the approximation.
* One can control the accuracy of integration using the `epsabs` and `epsrel` optional arguments and `quad` will try to return a result satisfying ``abs(I-result) <= max(epsabs, epsrel*abs(I))``, `I` denoting the (unknown) exact value of the integral.
* For functions depending on parameters, one can either use a lambda function or the `args=()` optional argument.
* `scipy.integrate.quad` can compute *some* improper integrals (see the discussion on accuracy later) using `np.inf` as one of the integration bounds.

In [2]:
def f(x):
    return np.sin(x)
print(scipy.integrate.quad(f,0,np.pi))

(2.0, 2.220446049250313e-14)


In [ ]:
# integrating a function depending on parameters using a lambda function or the args parameter. The later is the recommended way
def g(x,a):
    return np.sinc(x/a)

X = np.linspace(-4*np.pi,4*np.pi,201)
fig, ax = plt.subplots()
for a in (1, 2, 3):
    ax.plot(X,g(X,a), label = f'a={a}')
ax.legend()
ax.grid()

# for a in (1,2,3):
#     I1, err1 = scipy.integrate.quad(lambda x : g(x,a), 0, np.pi)
#     print(I1, err1)
#     I2, err2 = scipy.integrate.quad(g,0,np.pi, args=(a))
#     print(I2, err2)

In [ ]:
# Computing improper integrals
def f1(x):
    return 1/x**2
def f2(x):
    return 1/np.sqrt(x)
print(scipy.integrate.quad(f1,1,np.inf))
print(scipy.integrate.quad(f2,0,1))

In [ ]:
# Example of quad rightfully failing to compute a diverging integral
def f3(x):
    return 1/x
print(scipy.integrate.quad(f3,1,np.inf))

In [ ]:
# An example of quad failing to compute a perfectly good (but slow converging) improper integral
def f4(x):
    return np.sin(x)**2/x**2

X = np.linspace(-4*np.pi,4*np.pi,201)
fig, ax = plt.subplots()
for a in (1, 2, 3):
    ax.plot(X,f4(X))
ax.grid()
# notice that 0 <= (sin(x)/x)**2 < 1/x**2 so that by the squeeze theorem, the integral of f4 from 1 to infinity should converge 
print(scipy.integrate.quad(f4,1,np.inf))

`quad` tries really hard to compute definite integrals, but it can fail in some situations. Take for example $f(x) = e^{-x^2}$. We have that 
$$\int_{-\infty}^\infty f(x)\, dx = \sqrt{\pi}.$$
We would therefore expect that when computing $\int_{-b}^b f(x)\, dx$, `quad` would return an increasingly better approximation of $\sqrt{\pi}$ as $b \to \infty$.

In [ ]:
def f5(x):
    return np.exp(-x**2)
for b in (1,10,100,10000):
    print(b,scipy.integrate.quad(f5,-b,b), np.sqrt(np.pi))


What happens is that the since $e^{-x^2}$ decays so quickly from $x=0$, the quadrature algorithm used by `quad` does not detect the fact that `f5` is not identically equal to 0.

## `scipy.integrate.trapezoid` and `scipy.integrate.simpson`

In many situations, one may want to estimate the integral of a function that is only known at some given points (this was the case in the Euler's method section of HW2).
`scipy.integrate` can help here too. It implements the trapezoidal rule (see HW2) and Simpson's rule, which relies on the same idea, but where the function is approximated by parabolas instead of segments, using Lagramnge interpolation (see HW2 again!)

* Basic calling sequence: `scipy.integrate.trapezoid(Y, X)` or `scipy.integrate.simpson(Y, X)`
* Note that in order to apply the trapezoidal or Simpson rule, one does not need the location of the sampling points, but only their spacing. For equi-distributed sampling points, one can pass `dx=` instead of `X` as an argument.

In [ ]:
n = 51
X = np.linspace(0,np.pi,n)
Y = np.sin(X)
print(scipy.integrate.trapezoid(Y,X))
print(scipy.integrate.simpson(Y,X))

dx = np.pi / (n-1)
print(scipy.integrate.trapezoid(Y,dx = dx))
print(scipy.integrate.simpson(Y,dx = dx))


The ability to pass non-equidistributed points can be used in two ways.
   - When the values of `Y` are not known at equi-distributed points.
   - In order to improve the accuracy of integration, when one knows something about the function to be integrated (recall the issue with $\int_0^b e^{-x^2}\, dx$ for large `b`).

In [ ]:
n = 5
nint = 100
Ith= np.sqrt(np.pi)/2
for n in range(n):
    print(f'\n Integrating exp(-x^2) on (0,{10**n})')
    Xlog = np.logspace(-9,n,nint)
    # np.logspace returns points whose log is equi-distributed (i.e. with spacing increasing exponentially)

    X = np.linspace(0,10**n,nint)

    print(f'* Simpson with {nint} logspaced sampling points')
    I = scipy.integrate.simpson(f5(Xlog),Xlog)
    print(f'  result: {I:3.5e}, error = {abs(I-Ith):3.5e}')

    print(f'* Simpson with {nint} equidistributed sampling points')
    I = scipy.integrate.simpson(f5(X),X)
    print(f'  result: {I:3.5e}, error = {abs(I-Ith):3.5e}')

    print(f'* scipy.integrate.quad')
    I,err = scipy.integrate.quad(f5,0,10**n)
    print(f'  result: {I:3.5e}, error = {abs(I-Ith):3.5e}')


### Application: Fourier series

In the early XIXth century, Joseph Fourier proved that "any" periodic function on an interval $(-p,p)$ can be represented by an infinite series of $cos$ and $sin$ of the form:
$$
f(x) = \frac{a_0}{2} + \sum_{n=1}^\infty a_n \cos\left(\tfrac{n\pi}{p}x\right) + b_n \sin \left(\tfrac{n\pi}{p}x\right),
$$
where 
$$ 
a_n = \frac{1}{p}\int_{-p}^p f(x) \cos\left(\tfrac{n\pi}{p}x\right)\, dx
$$
and 
$$ 
b_n = \frac{2}{p}\int_{-p}^p f(x) \sin\left(\tfrac{n\pi}{p}x\right)\, dx
$$
Let's illustrate this when $f$ is a periodic function on $[-1,1]$ such that 
$$
f(x) = \begin{cases}
1+x & \text{ if } x<0\\
1-x & \text{ if } x\ge0.
\end{cases}
$$

In [ ]:
def ff(x):
    if x <0:
        return 1+x
    else:
        return 1

f = np.vectorize(ff)

fig, ax = plt.subplots()
X = np.linspace(-2,2,101)
ax.plot(X,f(X), label = '$f$')
ax.spines[['left', 'bottom']].set_position('zero')
ax.spines[['top', 'right']].set_visible(False)
ax.legend()

In [ ]:
def FourierCoefs(f, p, N):
    ''' 
    Returns the N first coefficients of the Fourier expansion of f on (-p,p)
    '''
    a = np.empty(N)
    b = np.empty(N)
    for n in range(N):
        a[n] = scipy.integrate.quad(lambda x : np.cos(n * np.pi * x / p) * f(x), -p,p)[0] / p
        b[n] = scipy.integrate.quad(lambda x : np.sin(n * np.pi * x / p) * f(x), -p,p)[0] / p
        # note that b[0] = 0
    return a,b
def FourierEval(x, a, b, p, N):
    '''
    Evaluates the sum of the Fourier series on (-p,p) with coefficients a and b at x
    '''
    f = np.ones(len(X)) * a[0]/2
    if N > len(a):
        raise IndexError(f"Requested {N} coefficients but got only {len(a)}")
    for n in range(1,N):
        f += a[n] * np.cos(n*np.pi*x/p) + b[n] * np.sin(n*np.pi*x/p)
    return f


def plot(a,b,p,N,X):
    ax.cla()
    ax.plot(X,f(X), label = '$f$')
    ax.plot(X,FourierEval(X,a,b,p,N),label = f'$F_{{{N}}}$')
    ax.spines[['left', 'bottom']].set_position('zero')
    ax.spines[['top', 'right']].set_visible(False)
    ax.legend()

N = 3
p = 1
a,b = FourierCoefs(f,p,N)
print('a: ', a)
print('b: ', b)


FourierEval(X,a,b,p,1)

In [ ]:
from ipywidgets import interact
p = 1
Nmax = 50
X = np.linspace(-p,p,501)

a,b = FourierCoefs(f,p,Nmax)

fig, ax = plt.subplots()
ax.plot(X,f(X), label = '$f$')

def pplot(N):
    plot(a,b,p,N)
interact(lambda N: plot(a,b,p,N,X), N = (1,Nmax,1))
# interact(pplot, N = (1,Nmax,1))
